In [ ]:
import datetime 
import numpy as np
import mpmath as mp
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import copy
from sklearn.preprocessing import StandardScaler
%matplotlib qt

from matplotlib.dates import DayLocator, HourLocator, DateFormatter, drange
import os 
from os import listdir
from os.path import isfile, join

def get_directory(folder):
    files= [a for a in listdir(folder) if isfile(join(folder,a))]
    
    return files
import os 
from os import listdir
from os.path import isfile, join

"get directory 2.0"

def get_directory_contents(folder):
    items = os.listdir(folder)
    files = [item for item in items if os.path.isfile(os.path.join(folder, item))]
    folders = [item for item in items if os.path.isdir(os.path.join(folder, item))]
    
    return {'files': files, 'folders': folders}

In [ ]:
from datetime import datetime 

def search_the_day(fecha):
    # Fecha del primer día del año
    primer_dia_del_ano = datetime(fecha.year, 1, 1)

    # Calcula la diferencia entre las dos fechas
    diferencia = fecha - primer_dia_del_ano

    # Obtiene el número de días transcurridos
    dias_transcurridos = diferencia.days+1
    #dias_transcurridos = diferencia.days
    
    return dias_transcurridos
    

## ITERATION

### Upload the dataset with the dissipation events

In [5]:
#Importing data
path1="C:/Users/enclab01/Desktop/THESIS/codigos tesis/Eventos de Disipacion/dataframe events2.txt"
titles=['Time_left','Time_right','Left','Right', 'Minutes', 'Prominences']
data= pd.read_table(path1, header=None, names=titles, sep='\t') #,parse_dates=['Dates_and_Hours'])
data=data.iloc[np.arange(1,len(data)),:] #delete the first row
data=data.reset_index(drop=True)
data_dissipation=data

hours=[]
for i in np.arange(0,len(data_dissipation)):
    hours.append(float(data_dissipation['Minutes'][i])/60 )
data_dissipation['Hours']=hours


'Filtering the events with more of 4 hours of duration'
for i in np.arange(0,len(data_dissipation)):
    if float(data_dissipation['Hours'][i])>4:
        data_dissipation.drop(i, axis=0, inplace=True)
        
data_dissipation=data_dissipation.reset_index(drop=True)
print(data_dissipation)

             Time_left           Time_right   Left  Right Minutes  \
0  2019-09-15 10:21:00  2019-09-15 11:17:00  62180  62236    56.0   
1  2019-09-15 13:54:00  2019-09-15 17:51:00  62393  62630   237.0   
2  2019-09-15 18:08:00  2019-09-15 21:11:00  62647  62830   183.0   
3  2019-09-16 10:15:00  2019-09-16 14:06:00  63614  63845   231.0   
4  2019-09-16 17:26:00  2019-09-16 19:20:00  64045  64159   114.0   
5  2019-09-27 19:35:00  2019-09-27 21:11:00  80014  80110    96.0   
6  2019-09-27 23:36:00  2019-09-28 00:47:00  80255  80326    71.0   

          Prominences     Hours  
0  3.1472611919999993  0.933333  
1  2.4030522919999995  3.950000  
2   4.218159829999999  3.050000  
3         2.764901778  3.850000  
4   8.839189050000002  1.900000  
5           8.6504512  1.600000  
6  5.3903867000000005  1.183333  


In [11]:
'Iteration for each dissipation event'

'FALTA IMPLEMENTAR EL CASO EN QUE TENEMOS DATOS UN DIA ANTERIOR'

main_folder = "C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2019"

events1hours=[]
events2hours=[]
events3hours=[]
events4hours=[]
events5hours=[]

#for i in np.arange(1,6):
for i in np.arange(0,len(data_dissipation)):
    beginning=pd.to_datetime(data_dissipation['Time_left'][i])
    end=pd.to_datetime(data_dissipation['Time_right'][i])
    day1=search_the_day(beginning)
    day2=search_the_day(end)
    hour1= beginning.hour
    hour2= end.hour
    #print(day1)
    #print(day2)
    #print(beginning)
    #print(end)
    
    if day1==day2: #If we have data for the same day
        
    #-------------------------------------------------------------------------------------------------------------------------------------------
        try:

            main_folderr=main_folder+'/'+str(day1)
            #print('main folder',main_folder)
            list_of_files=get_directory(main_folderr)                     #  Entrega los archivos al interior de la carpeta
            
            #List with the paths
            newlist=[]
            for item in list_of_files:                                 #  Iteracion sobre cada elemento, elimina los archivos que no son de interes
                # Verificar si la cadena inicia con la letra "p"
                if item.startswith("p"):
                    newlist.append(main_folderr+'/'+str(item))
            print('va bien')
            #print(newlist)  

            'Iteracion for each hour of a day'
            FINAL_LIST=[]
            
            list_of_data_per_day=[] #list with all the data per hour per day
            list_of_hours=[] 
            for j in np.arange(0,len(newlist)):
            #for j in np.arange(5,6):

                #-----------------------------------------------------------------------------------------------
                'Retain the date of the measurement'
                dato=pd.read_table(newlist[j]) 
                dato=dato.iloc[np.arange(2,3)][' PTS'][2]


                numeros = dato.split()
                numeros_enteros = [int(numero) for numero in numeros]
                numeros = numeros_enteros
                from datetime import datetime
                ano = 2000 + numeros[0]  
                mes = numeros[1]
                dia = numeros[2]
                hora = numeros[3]
                minuto = numeros[4]
                segundo = numeros[5]

                fecha_hora = datetime(ano, mes, dia, hora, minuto, segundo)
                #print('fecha_hora',fecha_hora)
                #print('fecha_hora',fecha_hora.hour)
                list_of_hours.append(fecha_hora)
                #-----------------------------------------------------------------------------------------------

                'Wind data'

                nombrearchivo = newlist[j] # Name of the file
                titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
                data=pd.read_table(nombrearchivo, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
                data=data.iloc[np.arange(1,len(data)),:] #delete the first row
                data=data.reset_index(drop=True)

                'Eliminar nans'
                data=data.astype(object)
                data = data.dropna()
                data=data.reset_index(drop=True)

                'Drop the rest of dataframes if there is more than 1'

                for k in np.arange(0,len(data)):
                    if str(data['HT'][k])=='HT':
                        data=data.iloc[np.arange(0,k),:] 
                        break

                'Eliminar los 99999'
                for k in np.arange(0,len(data)):
                    if float(data['SPD'][k])==999999 or float(data['HT'][k])>=1.5:
                        data.drop(k, axis=0, inplace=True)

                data=data.reset_index(drop=True)


                'moving average'
                data2=data[['HT','SPD','DIR']]

                window_size = 2  # Tamaño de la ventana de media móvil

                # Aplicar filtro de media móvil
                data['SPDsmoothed'] = data2['SPD'].rolling(window=window_size, min_periods=1).mean()
                data['DIRsmoothed'] = data2['DIR'].rolling(window=window_size, min_periods=1).mean()

                list_of_data_per_day.append(data)

                #------------------------------------------------------------------------------------------------------------------------------   

            indexes=[]
            for k in np.arange(0,len(list_of_data_per_day)):
                if list_of_hours[k].hour<hour1:
                    indexes.append(0)
                else:
                    indexes.append(1)
            
            for k in np.arange(0,len(list_of_data_per_day)):
                if list_of_hours[k].hour>hour2 and indexes[k]!=0:
                    indexes[k]=0
            
            print(indexes)    
            
            PREFINAL_LIST=[]
            'Keeping just the dataframes related to the time interval'
            
            for k in np.arange(0,len(list_of_data_per_day)):
                if indexes[k]==1:
                    PREFINAL_LIST.append([list_of_hours2[k],list_of_data_per_day2[k]])
            #print(list_of_data2)
            print('va bien')
             
        
            
           
            print(PREFINAL_LIST)
            #print('len pre',len(PREFINAL_LIST))
            
            boool=False #to start the iteration
            for k in np.arange(0,len(PREFINAL_LIST)):
                print('k',k)
                print(boool)
                if boool==False:
                    if k!=len(PREFINAL_LIST)-1:
                    
                        date1=PREFINAL_LIST[k][0]
                        date2=PREFINAL_LIST[k+1][0]                
                        print('ta bien')
                        difference = date2 - date1
                        difference_in_minutes = difference.total_seconds() / 60
                        print('ta bien')
                        #print('difference',difference_in_minutes)

                        if difference_in_minutes<=60:
                            print('DENTRO DEL FOR')
                            DATA1=pd.DataFrame(PREFINAL_LIST[k][1])
                            DATA2=pd.DataFrame(PREFINAL_LIST[k+1][1])
#HAY QUE CAMBIAR ESTOOOOO                            
                            MEAN_DATA=DATA1
                            #MEAN_DATA=(DATA1 + DATA2)/2
                            #print('meandata',MEAN_DATA)
                            MEAN_DATE=(date1+date2)/2
                            
                            
                            
                            #INAL_LIST.append([[list_of_hours2[k],list_of_hours2[k+1]],MEAN_DATA])
                            FINAL_LIST.append([MEAN_DATE,MEAN_DATA])

                            boool=True
                            print('ta bien')
                        else:
                            FINAL_LIST.append([PREFINAL_LIST[k][0],PREFINAL_LIST[k][1]])
                            
                            
                       

                    else:
                        FINAL_LIST.append([PREFINAL_LIST[k][0],PREFINAL_LIST[k][1]])
                
                else:
                    boool=False
            
            #print('largo',len(FINAL_LIST))
            print('va bien')
            
            #-----------------------------------------------------------------------------------------------------------------------------------
            
            'Here we split the events in three categories, the ones that last 1,2 or 3 hours'
            if len(FINAL_LIST)==1:
                events1hours.append(FINAL_LIST)
            if len(FINAL_LIST)==2:
                events2hours.append(FINAL_LIST)
            if len(FINAL_LIST)==3:
                events3hours.append(FINAL_LIST)   
            if len(FINAL_LIST)==4:
                events4hours.append(FINAL_LIST)
            if len(FINAL_LIST)==5:
                events5hours.append(FINAL_LIST)     

                
        except:
            print('No wind data for that day or error')
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
    
    
    #----------------------------------------------------------------------------------------------------------------------------------------------
    
    elif day1!=day2:
        
        try:
            'This goes for the first day'
            main_folderr1=main_folder+'/'+str(day1)
            #print('main folder',main_folder)
            list_of_files1=get_directory(main_folderr1)                     #  Returns the files of the folder

            #List with the paths
            newlist1=[]
            for item in list_of_files1:                                 #  Iteracion over each element, drops the useless files
                # Verify if the chain starts with a letter "p"
                if item.startswith("p"):
                    newlist1.append(main_folderr1+'/'+str(item))

            'This goes for the second day'
            main_folderr2=main_folder+'/'+str(day2)
            list_of_files2=get_directory(main_folderr2)                     #  Ruturns the files of the folder

            #List with the paths
            newlist2=[]
            for item in list_of_files2:                                 #  Iteracion over each element, drops the useless files
                # Verify if the chain starts with a letter "p"
                if item.startswith("p"):
                    newlist2.append(main_folderr2+'/'+str(item))        
            
            
            print('newlist1',newlist1)
            print('newlist2',newlist2)
            
            FINAL_LIST=[] 
            
            #-----------------------------------------------------------------------------------------------------------------------------
            #DAY 1:
            
            'Iteracion for each hour of a day'

            list_of_data_per_day1=[] #list with all the data per hour per day
            list_of_hours1=[] 
            for j in np.arange(0,len(newlist1)):
                'Retain the date of the measurement'
                dato1=pd.read_table(newlist1[j]) 
                dato1=dato1.iloc[np.arange(2,3)][' PTS'][2]


                numeros1 = dato1.split()
                numeros_enteros1 = [int(numero) for numero in numeros1]
                numeros1 = numeros_enteros1
                from datetime import datetime
                ano1 = 2000 + numeros1[0]  
                mes1 = numeros1[1]
                dia1 = numeros1[2]
                hora1 = numeros1[3]
                minuto1 = numeros1[4]
                segundo1 = numeros1[5]

                fecha_hora1 = datetime(ano1, mes1, dia1, hora1, minuto1, segundo1)
                print('fecha_hora',fecha_hora1)
                print('fecha_hora',fecha_hora1.hour)
                list_of_hours1.append(fecha_hora1)
          
                'Wind data'
                nombrearchivo1 = newlist1[j] # Name of the file
                titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
                data1=pd.read_table(nombrearchivo1, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
                data1=data1.iloc[np.arange(1,len(data1)),:] #delete the first row
                data1=data1.reset_index(drop=True)

                'Eliminar nans'
                data1=data1.astype(object)
                data1=data1.dropna()
                data1=data1.reset_index(drop=True)

                'Drop the rest of dataframes if there is more than 1'
                for k in np.arange(0,len(data1)):
                    if str(data1['HT'][k])=='HT':
                        data1=data1.iloc[np.arange(0,k),:] 
                        break

                'Eliminar los 99999'
                for k in np.arange(0,len(data1)):
                    if float(data1['SPD'][k])==999999 or float(data1['HT'][k])>=1.5:
                        data1.drop(k, axis=0, inplace=True)
                data1=data1.reset_index(drop=True)

                'moving average'
                data12=data1[['HT','SPD','DIR']]
                window_size = 2  # Tamaño de la ventana de media móvil
                # Aplicar filtro de media móvil
                data1['SPDsmoothed'] = data12['SPD'].rolling(window=window_size, min_periods=1).mean()
                data1['DIRsmoothed'] = data12['DIR'].rolling(window=window_size, min_periods=1).mean()
                list_of_data_per_day1.append(data1)

            indexes1=[]
            for k in np.arange(0,len(list_of_data_per_day1)):
                if list_of_hours1[k].hour<hour1:
                    indexes1.append(0)
                else:
                    indexes1.append(1)
             
            print(indexes1)
            
            PREFINAL_LIST=[]
            'Keeping just the dataframes related to the time interval'
            list_of_data12=[]
            for k in np.arange(0,len(list_of_data_per_day1)):
                if indexes1[k]==1:
                     PREFINAL_LIST.append([list_of_hours1[k],list_of_data_per_day1[k]])
            
            

            #DAY 2:
            
            'Iteracion for each hour of a day'

            list_of_data_per_day2=[] #list with all the data per hour per day
            list_of_hours2=[] 
            for j in np.arange(0,len(newlist2)):
            #for j in np.arange(5,6):
                'Retain the date of the measurement'
                dato2=pd.read_table(newlist2[j]) 
                dato2=dato2.iloc[np.arange(2,3)][' PTS'][2]

                numeros2 = dato2.split()
                numeros_enteros2 = [int(numero) for numero in numeros2]
                numeros2 = numeros_enteros2
                from datetime import datetime
                ano2 = 2000 + numeros2[0]  
                mes2 = numeros2[1]
                dia2 = numeros2[2]
                hora2 = numeros2[3]
                minuto2 = numeros2[4]
                segundo2 = numeros2[5]

                fecha_hora2 = datetime(ano2, mes2, dia2, hora2, minuto2, segundo2)
                print('fecha_hora',fecha_hora2)
                print('fecha_hora',fecha_hora2.hour)
                list_of_hours2.append(fecha_hora2)
                
                'Wind data'

                nombrearchivo2 = newlist2[j] # Name of the file
                titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
                data2=pd.read_table(nombrearchivo2, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
                data2=data2.iloc[np.arange(1,len(data2)),:] #delete the first row
                data2=data2.reset_index(drop=True)
                
                
                
                'Eliminar nans'
                data2=data2.astype(object)
                data2=data2.dropna()
                data2=data2.reset_index(drop=True)

                'Drop the rest of dataframes if there is more than 1'

                for k in np.arange(0,len(data2)):
                    if str(data2['HT'][k])=='HT':
                        data2=data2.iloc[np.arange(0,k),:] 
                        break

                'Eliminar los 99999'
                for k in np.arange(0,len(data2)):
                    if float(data2['SPD'][k])==999999 or float(data2['HT'][k])>=1.5:
                        data2.drop(k, axis=0, inplace=True)

                data2=data2.reset_index(drop=True)

                
                'moving average'
                data22=data2[['HT','SPD','DIR']]

                window_size = 2  # Tamaño de la ventana de media móvil

                # Aplicar filtro de media móvil
                data2['SPDsmoothed'] = data22['SPD'].rolling(window=window_size, min_periods=1).mean()
                data2['DIRsmoothed'] = data22['DIR'].rolling(window=window_size, min_periods=1).mean()
                
                list_of_data_per_day2.append(data2)

           
            indexes2=[]
            for k in np.arange(0,len(list_of_data_per_day2)):
                if list_of_hours2[k].hour<=hour2:
                    indexes2.append(1)
                else:
                    indexes2.append(0)
             
            print('indexes2', indexes2)
            
            'Keeping just the dataframes related to the time interval'
            for k in np.arange(0,len(list_of_data_per_day2)):
                if indexes2[k]==1:
                    PREFINAL_LIST.append([list_of_hours2[k],list_of_data_per_day2[k]])
                                

            print('len pre',len(PREFINAL_LIST))
            
            boool=False #to start the iteration
            for k in np.arange(0,len(PREFINAL_LIST)):
                print('k',k)
                
                if boool==False:
                    if k!=len(PREFINAL_LIST)-1:
                    
                        date1=PREFINAL_LIST[k][0]
                        date2=PREFINAL_LIST[k+1][0]                

                        difference = date2 - date1
                        difference_in_minutes = difference.total_seconds() / 60
                        print('difference',difference_in_minutes)

                        if difference_in_minutes<=60:
                            DATA1=PREFINAL_LIST[k][1]
                            DATA2=PREFINAL_LIST[k][1]
                            MEAN_DATA=(DATA1 + DATA2)/2
                            MEAN_DATE=(date1+date2)/2
                            #INAL_LIST.append([[list_of_hours2[k],list_of_hours2[k+1]],MEAN_DATA])
                            FINAL_LIST.append([MEAN_DATE,MEAN_DATA])

                            boool=True
                        else:
                            FINAL_LIST.append([PREFINAL_LIST[k][0],PREFINAL_LIST[k][1]])
                            
                    

                    else:
                        FINAL_LIST.append([PREFINAL_LIST[k][0],PREFINAL_LIST[k][1]])
                
                else:
                    boool=False
            
            print('largo',len(FINAL_LIST))
           
        
            #-----------------------------------------------------------------------------------------------------------------------------------
            
            'Here we split the events in three categories, the ones that last 1,2 or 3 hours'
            if len(FINAL_LIST)==1:
                events1hours.append(FINAL_LIST)
            if len(FINAL_LIST)==2:
                events2hours.append(FINAL_LIST)
            if len(FINAL_LIST)==3:
                events3hours.append(FINAL_LIST)   
            if len(FINAL_LIST)==4:
                events4hours.append(FINAL_LIST) 
            if len(FINAL_LIST)==5:
                events5hours.append(FINAL_LIST)     
            
            
            
            'PLOT'
            plt.figure()
            for k in np.arange(0,len(FINAL_LIST)):
                               
                x=FINAL_LIST[k][1]['HT'].astype(float)
                x=x.round(decimals=1, out=None)
                windspeed=FINAL_LIST[k][1]['SPD'].astype(float)
                windspeedmean=FINAL_LIST[k][1]['SPDsmoothed'].astype(float)
                windangles=FINAL_LIST[k][1]['DIR'].astype(float)
                windanglesmean=FINAL_LIST[k][1]['DIRsmoothed'].astype(float)
                ysize=15
                #plt.figure()
                ax1=plt.subplot(211)
                plt.plot(x,windspeed,'x-',c='black')
                #plt.plot(x,windspeedmean,'-',c='red')
                ax1.tick_params(axis="y", labelsize=ysize, labelcolor='black')
                ax1.tick_params(axis="x", labelsize=ysize, labelcolor='black')  
                plt.ylabel('Wind speed (m/s)',fontsize=ysize,color='black')  
                plt.setp(ax1.get_xticklabels(), visible=False)
                          
                ax2=plt.subplot(212,sharex=ax1)
                plt.plot(x,windangles,'x-',c='purple')
                #plt.plot(x,windanglesmean,'-',c='blue')
                ax2.tick_params(axis="y", labelsize=ysize, labelcolor='black')  
                ax2.tick_params(axis="x", labelsize=ysize, labelcolor='black') 
                plt.ylabel('Wind direction (deg)',fontsize=ysize,color='black')
                plt.xlabel('Height above ground (km)',fontsize=ysize,color='black')                
                #axx1=ax1.twinx()
                #plt.plot(x,windangles,'x-',c='purple')
                #plt.plot(x,windanglesmean,'-',c='blue')
                #axx1.tick_params(axis="y", labelsize=ysize, labelcolor='purple')  
                #plt.ylabel('Wind direction (deg)',fontsize=ysize,color='purple')


                
        except:
            print('No wind data for that day or error')

va bien
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
va bien
[]
va bien
va bien
[1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
va bien
[[datetime.datetime(2019, 9, 28, 0, 0, 5),        HT  SPD  DIR MET_QC  RAD1  RAD2  RAD3 CNT1 CNT2 CNT3 SNR1 SNR2 SNR3  \
0   0.195  2.9  331      0   0.2   0.1  -0.7   11   11   11    3   11    7   
1   0.297  6.3  327      0   0.4   0.1  -1.6   11   11   11   13   15   17   
2   0.398  5.2  324      0   0.4   0.0  -1.4   11   11   11   18   16   22   
3   0.499  4.4  315      0   0.4  -0.2  -1.1   11   11   11   12   10   16   
4   0.601  3.8  313      0   0.4  -0.2  -1.0   11   11   11    6    6   12   
5   0.702  2.9  321      0   0.2  -0.1  -0.7   11   11   11    3    8    9   
6   0.803  2.4  329      0   0.1   0.0  -0.6   11   11   11    4   15   14   
7   0.905  3.4  328      0   0.0   0.1  -0.9   11   11   11    5   13   13   
8   1.006  3.8  319      0  -0.1  -0.1  -1.0   11   11   11    0    1   11   
9   1.107  2.7  315      0  -0.2  -0.1  -0.7   11   11   11   -6    1   

In [7]:
import pandas as pd

# Supongamos que tienes dos DataFrames df1 y df2 de igual forma (4x10).

# Crear DataFrames de ejemplo
data1 = {'A': [1, 2, 3, 4],
         'B': [5, 6, 7, 8],
         'C': [9, 10, 11, 12],
         'D': [13, 14, 15, 16],
         'E': [17, 18, 19, 20],
         'F': [21, 22, 23, 24],
         'G': [25, 26, 27, 28],
         'H': [29, 30, 31, 32],
         'I': [33, 34, 35, 36],
         'J': [37, 38, 39, 40]}

data2 = {'A': [41, 42, 43, 44],
         'B': [45, 46, 47, 48],
         'C': [49, 50, 51, 52],
         'D': [53, 54, 55, 56],
         'E': [57, 58, 59, 60],
         'F': [61, 62, 63, 64],
         'G': [65, 66, 67, 68],
         'H': [69, 70, 71, 72],
         'I': [73, 74, 75, 76],
         'J': [77, 78, 79, 80]}

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Calcular el promedio de los DataFrames
df_promedio = (df1 + df2) / 2

# df_promedio contendrá el valor promedio de los dos DataFrames
print(df_promedio)

      A     B     C     D     E     F     G     H     I     J
0  21.0  25.0  29.0  33.0  37.0  41.0  45.0  49.0  53.0  57.0
1  22.0  26.0  30.0  34.0  38.0  42.0  46.0  50.0  54.0  58.0
2  23.0  27.0  31.0  35.0  39.0  43.0  47.0  51.0  55.0  59.0
3  24.0  28.0  32.0  36.0  40.0  44.0  48.0  52.0  56.0  60.0


### PLOTS

In [12]:
 'PLOT 4 horas'

for a in np.arange(0,4):
    plt.figure()
    for k in np.arange(0,len(events4hours)):
        print(k)


        x=events4hours[k][a][1]['HT'].astype(float)
        x=x.round(decimals=1, out=None)
        windspeed=events4hours[k][a][1]['SPD'].astype(float)
        windspeedmean=events4hours[k][a][1]['SPDsmoothed'].astype(float)
        windangles=events4hours[k][a][1]['DIR'].astype(float)
        windanglesmean=events4hours[k][a][1]['DIRsmoothed'].astype(float)
        ysize=15

        plt.title('Hour nº'+str(a+1))


        ax1=plt.subplot(211)
        plt.plot(x,windspeed,'x-',c='black')
                        #plt.plot(x,windspeedmean,'-',c='red')
        ax1.tick_params(axis="y", labelsize=ysize, labelcolor='black')
        ax1.tick_params(axis="x", labelsize=ysize, labelcolor='black')  
        plt.ylabel('Wind speed (m/s)',fontsize=ysize,color='black')  
        plt.setp(ax1.get_xticklabels(), visible=False)

        ax2=plt.subplot(212,sharex=ax1)
        plt.plot(x,windangles,'x-',c='purple')
                        #plt.plot(x,windanglesmean,'-',c='blue')
        ax2.tick_params(axis="y", labelsize=ysize, labelcolor='black')  
        ax2.tick_params(axis="x", labelsize=ysize, labelcolor='black') 
        plt.ylabel('Wind direction (deg)',fontsize=ysize,color='black')
        plt.xlabel('Height above ground (km)',fontsize=ysize,color='black') 

0
1
0
1
0
1
0


C:\Users\enclab01\AppData\Local\Temp\ipykernel_11468\1874962435.py:20: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  ax1=plt.subplot(211)


1


In [ ]:

main_folder = "C:/Users/enclab01/Desktop/THESIS/Datos Viento/WwWind/pts/2022"
# Lista de archivos en la carpeta principal
files_in_main_folder = os.listdir(main_folder)


#hora1=[]
fechas=[]
list_of_data=[]


#for i in np.arange(0,len(files_in_main_folder)):             # Para cada carpeta dentro del año
for i in np.arange(1,2):    
    
    subfolder=main_folder+'/'+str(files_in_main_folder[i])
    list_of_files=get_directory(subfolder)                     #  Entrega los archivos al interior de la carpeta

    #List with the paths
    newlist=[]
    for item in list_of_files:                                 #  Iteracion sobre cada elemento, elimina los archivos que no son de interes
        # Verificar si la cadena inicia con la letra "p"
        if item.startswith("p"):
            newlist.append(subfolder+'/'+str(item))

    print(newlist)  
    
    'Iteracion for each hour of a day'
    
    list_of_data_per_day=[] #list with all the data per hour per day
    
    for j in np.arange(0,len(newlist)):
    #for j in np.arange(5,6):
        
        #-----------------------------------------------------------------------------------------------
        'Retain the date of the measurement'
        dato=pd.read_table(newlist[j]) 

        print(dato.iloc[np.arange(2,3)][' PTS'][2])
        dato=dato.iloc[np.arange(2,3)][' PTS'][2]

        
        numeros = dato.split()
        numeros_enteros = [int(numero) for numero in numeros]
        numeros = numeros_enteros
        from datetime import datetime
        ano = 2000 + numeros[0]  
        mes = numeros[1]
        dia = numeros[2]
        hora = numeros[3]
        minuto = numeros[4]
        segundo = numeros[5]

        fecha_hora = datetime(ano, mes, dia, hora, minuto, segundo)
        print(fecha_hora)
        fechas.append(fecha_hora)
        #-----------------------------------------------------------------------------------------------

        'Wind data'
        
        nombrearchivo = newlist[j] # Name of the file
        titles=['HT',      'SPD',      'DIR',    'MET_QC'  ,'RAD1',      'RAD2',      'RAD3',      'CNT1',      'CNT2',      'CNT3',      'SNR1',      'SNR2',      'SNR3', 'QC1', 'QC2', 'QC3'   ]
        data=pd.read_table(nombrearchivo, skiprows=10 ,header=None,names=titles, sep='\s+') #,parse_dates=['t_utc'])
        data=data.iloc[np.arange(1,len(data)),:] #delete the first row
        data=data.reset_index(drop=True)
        
        'Eliminar nans'
        data=data.astype(object)
        data = data.dropna()
        data=data.reset_index(drop=True)
        
        
        
        'Drop the rest of dataframes if there is more than 1'
        
        for i in np.arange(0,len(data)):
            if str(data['HT'][i])=='HT':
                data=data.iloc[np.arange(0,i),:] 
                break
        
        'Eliminar los 99999'
        for i in np.arange(0,len(data)):
            if float(data['SPD'][i])==999999 or float(data['HT'][i])>=1.5:
                data.drop(i, axis=0, inplace=True)
            
        data=data.reset_index(drop=True)
        
        
        'moving average'
        data2=data[['HT','SPD','DIR']]


        window_size = 2  # Tamaño de la ventana de media móvil

        # Aplicar filtro de media móvil
        data['SPDsmoothed'] = data2['SPD'].rolling(window=window_size, min_periods=1).mean()
        data['DIRsmoothed'] = data2['DIR'].rolling(window=window_size, min_periods=1).mean()
        
        list_of_data_per_day.append(data)
        
        
        #'ver que pasa en el kilometro 0.8'
        
        #data3=data.copy()
        
        #for i in np.arange(0,len(data)):
         #   if float(data['HT'][i]) <=0.7 or float(data['HT'][i])>=0.9:
          #      data3.drop(i, axis=0, inplace=True)
        
        
        #mean=np.mean(data3['SPD'].astype(float))
        #print(mean)
        #hora1.append(mean)
        
        'PLOT'
        
        #x=data['HT'].astype(float)
        #x=x.round(decimals=1, out=None)
        #windspeed=data['SPD'].astype(float)
        #windspeedmean=data['SPDsmoothed'].astype(float)
        #windangles=data['DIR'].astype(float)
        #windanglesmean=data['DIRsmoothed'].astype(float)
        
        #ysize=15
        
        #plt.figure()
        #ax1=plt.subplot(1,1,1)
        #plt.plot(x,windspeed,'x-',c='black')
        #plt.plot(x,windspeedmean,'-',c='red')
        #ax1.tick_params(axis="y", labelsize=ysize, labelcolor='black')
        #ax1.tick_params(axis="x", labelsize=ysize, labelcolor='black')  
        #plt.ylabel('Wind speed (m/s)',fontsize=ysize,color='black')  
        #plt.xlabel('Height above ground (km)',fontsize=ysize,color='black') 
       

        #axx1=ax1.twinx()
        #plt.plot(x,windangles,'x-',c='purple')
        #plt.plot(x,windanglesmean,'-',c='blue')
        #axx1.tick_params(axis="y", labelsize=ysize, labelcolor='purple')  
        #plt.ylabel('Wind direction (deg)',fontsize=ysize,color='purple')
    
    list_of_data.append(list_of_data_per_day)   
        
        
        
        
        
        
        
        
        
    


In [ ]:
print(list_of_data[0][23])